# Cadeia de Markov

In [ ]:
###########################################
### Sistemas Baseados em Conhecimento
### Notebook: N03 (Modelo Oculto de Markov)   
###########################################

#https://www.datacamp.com/tutorial/markov-chains-python-tutorial

In [3]:
# !pip install pomegranate==v0.15
# !pip install gdown

In [10]:
import os
import gdown
import numpy as np
import pandas as pd
from pomegranate import *

In [18]:
# O ID do arquivo
file_id = '1Ufc4cwvjOUYHNq3fly77FPQ9F10BMDH1'

# URL para download direto
url = f'https://drive.google.com/uc?id={file_id}'

# Cria a pasta de destino se ela não existir
folder_path = '/content/Datasets/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Define o caminho completo para o arquivo de saída
output_path = os.path.join(folder_path, 'dataset') # Nome do arquivo alterado

# Baixa o arquivo usando a URL
gdown.download(url, output_path, quiet=False)

# Confirma que o arquivo foi baixado
print(f'Arquivo baixado com sucesso em: {output_path}')

# Lê o arquivo
df = pd.read_csv(output_path)

# Exibe as primeiras linhas do DataFrame
coluna_umidade = df["umidade"]
coluna_umidade.head()

Downloading...
From: https://drive.google.com/uc?id=1Ufc4cwvjOUYHNq3fly77FPQ9F10BMDH1
To: c:\content\Datasets\dataset
100%|██████████| 9.23k/9.23k [00:00<00:00, 8.89MB/s]

Arquivo baixado com sucesso em: /content/Datasets/dataset


0    75
1    76
2    75
3    76
4    76
Name: umidade, dtype: int64

In [12]:
coluna_umidade = df["umidade"]
coluna_umidade.head()

0    75
1    76
2    75
3    76
4    76
Name: umidade, dtype: int64

In [6]:
def umidade(u):
    if u < 40:
        return "baixa"
    elif u < 70:
        return "media"
    else:
        return "alta"

In [7]:
df["state"] = df["umidade"].apply(umidade)
print("Sequência de estados (umidade):")
print(df[["umidade", "state"]])


Sequência de estados (umidade):
      umidade state
0          75  alta
1          76  alta
2          75  alta
3          76  alta
4          76  alta
...       ...   ...
2245       76  alta
2246       76  alta
2247       79  alta
2248       78  alta
2249       79  alta

[2250 rows x 2 columns]


In [8]:
sequence = df["state"].tolist()
states = ["baixa", "media", "alta"]

# Construir matriz de transição
counts = pd.crosstab(index=sequence[:-1], columns=sequence[1:])
transitions = (counts.T / counts.T.sum()).fillna(0).T.reindex(index=states, columns=states, fill_value=0).values
print(transitions)

[[0.         0.         0.        ]
 [0.         0.85962733 0.14037267]
 [0.         0.07825485 0.92174515]]


In [9]:
# Distribuição inicial
start_probs = (pd.Series(sequence).value_counts(normalize=True).reindex(states, fill_value=0).to_dict())
start = DiscreteDistribution(start_probs)


transition_table = []
for i_idx, i in enumerate(states):
    for j_idx, j in enumerate(states):
        transition_table.append([i, j, transitions[i_idx, j_idx]])

transitions_pomegranate = ConditionalProbabilityTable(transition_table, [start])
# Criar Cadeia de Markov
model = MarkovChain([start, transitions_pomegranate])

print("\nSimulação de 30 passos:")
print(model.sample(30))


Simulação de 30 passos:
['alta', 'alta', 'alta', 'alta', 'alta', 'alta', 'alta', 'alta', 'alta', 'alta', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'media', 'alta', 'alta', 'alta', 'alta', 'alta', 'alta']
